In [1]:
pip install yfinance pandas numpy


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
      --------------------------------------- 0.0/3.0 MB 653.6 kB/s eta 0:00:05
     --- ------------------------------------ 0.3/3.0 MB 2.9 MB/s eta 0:00:01
     ------- -------------------------------- 0.6/3.0 MB 3.9 MB/s eta 0:00:01
     ----------- ---------------------------- 0.9/3.0 MB 4.7 MB/s eta 0:00:01
     --------------- ------------------------ 1.2/3.0 MB 5.1 MB/s eta 0:00:01
     -------------------- ------------------- 1.5/3.0 MB 5.7 MB/s eta 0:00:01
     ------------------------ --------------- 1.8/3.0 MB 5.6 MB/s eta 0:00:01
     ------------------------------ --------- 2.3/3.0 MB 6.2 MB/s eta 0:00:01
     ------------------------------------- -- 2.8/3.0 MB 6.7 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 6.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependenc

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import yfinance as yf
import pandas as pd

# Download NIFTY50 data (symbol: ^NSEI)
data = yf.download("^NSEI", start="2022-01-01", end="2024-12-31", interval="1d")
data = data.dropna()  # Remove missing values
data.reset_index(inplace=True)
print(data.head())


[*********************100%***********************]  1 of 1 completed

Price        Date         Close          High           Low          Open  \
Ticker                    ^NSEI         ^NSEI         ^NSEI         ^NSEI   
0      2022-01-03  17625.699219  17646.650391  17383.300781  17387.150391   
1      2022-01-04  17805.250000  17827.599609  17593.550781  17681.400391   
2      2022-01-05  17925.250000  17944.699219  17748.849609  17820.099609   
3      2022-01-06  17745.900391  17797.949219  17655.550781  17768.500000   
4      2022-01-07  17812.699219  17905.000000  17704.550781  17797.599609   

Price   Volume  
Ticker   ^NSEI  
0       200500  
1       247400  
2       251500  
3       236500  
4       239300  


In [27]:
data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]


In [28]:
def bollinger_bands(df, window=20, num_std=2):
    df['BB_Middle'] = df['Close'].rolling(window).mean()
    df['BB_Std'] = df['Close'].rolling(window).std()
    df['BB_Upper'] = df['BB_Middle'] + num_std * df['BB_Std']
    df['BB_Lower'] = df['BB_Middle'] - num_std * df['BB_Std']
    return df


In [29]:
def macd(df, fast=12, slow=26, signal=9):
    df['EMA_fast'] = df['Close'].ewm(span=fast, adjust=False).mean()
    df['EMA_slow'] = df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    return df


In [30]:
def stochastic_oscillator(df, k_window=14, d_window=3):
    df['Low_Min'] = df['Low'].rolling(window=k_window).min()
    df['High_Max'] = df['High'].rolling(window=k_window).max()
    df['%K'] = 100 * (df['Close'] - df['Low_Min']) / (df['High_Max'] - df['Low_Min'])
    df['%D'] = df['%K'].rolling(window=d_window).mean()
    return df


In [34]:
def vwap(df):
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3  # Typical Price
    df['Cumulative_TP_Volume'] = (df['TP'] * df['Volume']).cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df


In [35]:
def rsi(df, window=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df


In [36]:
def rsi(df, window=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df


In [38]:
def atr(df, window=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window).mean()
    return df


In [39]:
df = data.copy()
df = bollinger_bands(df)
df = macd(df)
df = stochastic_oscillator(df)
df = vwap(df)
df = rsi(df)
df = atr(df)

print(df.tail())


          Date         Close          High           Low          Open  \
733 2024-12-23  23753.449219  23869.550781  23647.199219  23738.199219   
734 2024-12-24  23727.650391  23867.650391  23685.150391  23769.099609   
735 2024-12-26  23750.199219  23854.500000  23653.599609  23775.800781   
736 2024-12-27  23813.400391  23938.849609  23800.599609  23801.400391   
737 2024-12-30  23644.900391  23915.349609  23599.300781  23796.900391   

     Volume     BB_Middle      BB_Std      BB_Upper      BB_Lower  ...  \
733  189800  24339.255078  339.650109  25018.555295  23659.954861  ...   
734  177700  24315.912598  365.202940  25046.318478  23585.506717  ...   
735  177700  24289.677539  386.528042  25062.733624  23516.621454  ...   
736  176800  24284.640039  392.293283  25069.226605  23500.053473  ...   
737  364900  24260.330078  416.619165  25093.568409  23427.091748  ...   

               TP  Cumulative_TP_Volume  Cumulative_Volume          VWAP  \
733  23756.733073          4.18659